In [1]:
import os

os.environ['TIKA_LOG_PATH'] = os.getcwd()

In [11]:
from glob import glob
from tika import parser
from bs4.element import Comment
from bs4 import BeautifulSoup

TIKA_HOST = 'http://localhost:9998'

In [3]:
pdfs = glob('*/*.pdf')

In [40]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body, is_html=True):
    if is_html:
        soup = BeautifulSoup(body, 'html.parser')
    else:
        soup = body
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)
    return '\n'.join(t.strip() for t in visible_texts).strip()

In [41]:
def parse_tika(filename):
    raw_xml = parser.from_file(filename, TIKA_HOST, xmlContent=True)
    body = BeautifulSoup(raw_xml['content']).find('body')
    t = []
    for b in body:
        try:
            t_ = text_from_html(b, is_html=False)
            t.append(t_)
        except Exception as e:
            pass
    return t

In [50]:
!rm -rf json
!mkdir json

In [53]:
from tqdm import tqdm
import json

for f in tqdm(pdfs):
    filename = os.path.join('json', f.replace('/', '-') + '.json')
    if os.path.exists(filename):
        continue
    
    try:
        b = parse_tika(f)
        with open(filename, 'w') as fopen:
            json.dump(b, fopen)
    except Exception as e:
        print(e)

 21%|████████▍                              | 486/2262 [00:03<00:13, 129.14it/s]

object of type 'NoneType' has no len()


 23%|█████████                               | 514/2262 [00:18<01:50, 15.82it/s]

object of type 'NoneType' has no len()


 97%|█████████████████████████████████████▋ | 2186/2262 [18:59<00:39,  1.93it/s]

object of type 'NoneType' has no len()


100%|███████████████████████████████████████| 2262/2262 [19:54<00:00,  1.89it/s]


In [54]:
!du -hs json

454M	json
